In [19]:
import pandas as pd
import yfinance as yf
import pymc as pm
import arviz as az
import math
import scipy.stats as stats
from scipy.stats import norm
import numpy as np

## Grab Data

In [ ]:
# Download stock data
data = yf.download('AAPL', start='2024-01-01')

# Calculate returns
data['aapl_r'] = data['Close'].pct_change()

# Select last 200 rows of Close and aapl_r columns
stocks = data[['Close', 'aapl_r']].tail(200)

# Define the data
n = len(stocks)
y = stocks['aapl_r'].values

[*********************100%***********************]  1 of 1 completed


## Model

In [12]:
# Define the PyMC model
with pm.Model() as model:
    # Priors
    mu = pm.Normal('mu', mu=0, sigma=100)
    sigma2 = pm.InverseGamma('sigma2', alpha=0.001, beta=0.001)
    # Likelihood
    y_lik = pm.Normal('y_lik', mu=mu, sigma=pm.math.sqrt(sigma2), observed=y)
    # Sample posterior
    trace = pm.sample(1000, tune=500)

az.summary(trace)

Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [mu, sigma2]


Output()

Sampling 4 chains for 500 tune and 1_000 draw iterations (2_000 + 4_000 draws total) took 20 seconds.


,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
mu,0.001,0.001,-0.001,0.003,0.0,0.0,3847.0,2802.0,1.0
sigma2,0.000,0.000,0.000,0.000,0.0,0.0,4329.0,2587.0,1.0


## Using the posterior distribution for the VaR0.01, find the posterior mean and a 95% probability interval for this quantity (using quantiles of the posterior distribution...not hdi).

In [20]:
# Compute z-score for 1% quantile
z_01 = norm.ppf(0.01)  # ≈ -2.33

# Extract posterior samples
posterior = az.extract(trace)
mu_samples = posterior["mu"]
sigma_samples = np.sqrt(posterior["sigma2"])

# Compute VaR_0.01
var_01_samples = mu_samples + z_01 * sigma_samples

# Compute mean and 95% probability interval
var_01_mean = var_01_samples.mean()
var_01_interval = np.quantile(var_01_samples, [0.025, 0.975])

# Print results
print(f"Posterior mean of VaR_0.01: {var_01_mean:.4f}")
print(f"95% probability interval: [{var_01_interval[0]:.4f}, {var_01_interval[1]:.4f}]")

Posterior mean of VaR_0.01: -0.0363
95% probability interval: [-0.0408, -0.0322]


## Let's say you have 1000 shares of Apple stock.  Using your VaR posterior distribution for ROC, what is the mean of the posterior distribution for the VaR for the value of the portfolio tomorrow. 

In [35]:
stocks['Close'].iloc[-1] * 1000 * float(var_01_mean)

Ticker
AAPL   -7783.173213
Name: 2025-03-17 00:00:00, dtype: float64

## From the previous question, find a 95% probability interval for the VaR0.01 for the value of this portfolio.

In [38]:
# lowwer bound
print(stocks['Close'].iloc[-1] * 1000 * float(var_01_interval[0]))

#upper bound
print(stocks['Close'].iloc[-1] * 1000 * float(var_01_interval[1]))

Ticker
AAPL   -8748.477544
Name: 2025-03-17 00:00:00, dtype: float64
Ticker
AAPL   -6905.00998
Name: 2025-03-17 00:00:00, dtype: float64
